## Session 6


Convolutional Autoencoder

In [1]:
import torch
import torchvision.datasets as dsets
import torch.nn as nn
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
from model import Encoder, Decoder, LogisticRegression
from evaluator import ModelEvaluator, AutoEncoderEvaluator

## Use latent features to train and test a simple MLP

In [2]:
def load_model(model_name, key='state_dict_encoder'):
    model_dir = '../Session6/model/' + model_name
    checkpoint = torch.load(model_dir)
    checkpoint = checkpoint[key]
    return checkpoint

In [ ]:
# Set Hyperparameters
epochs = 10
batch_size = 10
lr = 0.001
# Model
l2 = 0.0
optim = 'adam'
encoder_epoch = 8
use_gpu= True
model_name = 'AutoEncoder_lr_{}_opt_{}_epoch_{}'.format(lr, optim, encoder_epoch)

encoder = Encoder(batch_size=batch_size)
decoder = Decoder(batch_size=batch_size)
if use_gpu:
    encoder = encoder.cuda()
    decoder = decoder.cuda()

encoder.load_state_dict(load_model(model_name, key='state_dict_encoder'))
decoder.load_state_dict(load_model(model_name, key='state_dict_decoder'))

In [ ]:
encoder.eval()

In [ ]:
decoder.eval()

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
trainset = dsets.CIFAR10('./data', train=True, download=True, transform=transforms.ToTensor())
testset = dsets.CIFAR10('./data', train=False, download=True, transform=transforms.ToTensor())
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=3)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=3)

def plot_img(tensor, num_cols=10):
  num_imgs = tensor.shape[0]
  num_rows = 1+ num_imgs // num_cols
  fig = plt.figure(figsize=(num_cols,num_rows))
  for i in range(num_imgs):
    ax1 = fig.add_subplot(num_rows,num_cols,i+1)
    ax1.imshow(tensor[i])
    ax1.axis('off')
    ax1.set_xticklabels([])
    ax1.set_yticklabels([])

  plt.subplots_adjust(wspace=0.1, hspace=0.1)
  plt.show() 

for images, labels in testloader:
    output = encoder.forward(images.cuda())
    output = decoder.forward(output)
    break

plot_img(images.detach().cpu().numpy())    
'''
img = images[0].cpu()
print(img.shape)
output_img = output[0].cpu()
origin = img.data.numpy().transpose((1, 2, 0))
out = output_img.data.numpy().transpose((1, 2, 0))

plt.imshow(origin)
plt.show()

plt.imshow(out)
plt.show()

print(labels[0])
'''

In [ ]:


n_in = 32768  # 512x8x8
n_hidden = 128
n_out = 10
model_epochs = 20
model_lr = 0.001
l2 = 0.0
#train classifier
model = LogisticRegression(n_in, n_hidden, n_out)
modeleval = ModelEvaluator(model, model_epochs, model_lr, l2=l2, use_gpu=True, optim='adam')
acc_ = modeleval.evaluator(encoder, trainloader, testloader, noise=add_noise, print_every=100)
'''
for b_idx, (train_data, train_labels) in enumerate(trainloader):
    if add_noise:
            train_data_n = torch.mul(train_data+0.25, 0.1 * torch.rand(batch_size, 3, 32, 32))
    if use_gpu:
        train_data = train_data.cuda(non_blocking=True)
        train_labels = train_labels.cuda()

    if add_noise:
        latent_repr = encoder.forward(train_data_n)
    else:
        latent_repr = encoder.forward(train_data)
    print(latent_repr.shape)
'''

In [ ]:
modeleval.plot_loss()

## Denoising AutoEncoder

Add noise to encoder

In [ ]:
# Set Hyperparameters
epochs = 10
batch_size = 50
lr = 0.001
# Model
l2 = 0.0
optim = 'adam'
model_name = 'AutoEncoder_lr_{}_opt_{}_epoch_{}'.format(lr, optim, encoder_epoch)
model_name = model_name + '_dae'

encoder = Encoder(batch_size=batch_size)
encoder_epoch = 10
encoder.load_state_dict(load_encoder(model_name))


n_in = 32768  # 512x8x8
n_hidden = 512
n_out = 10
model_epochs = 20
model_lr = 0.001
l2 = 0.0
#train classifier
model = LogisticRegression(n_in, n_hidden, n_out)
modeleval = ModelEvaluator(model, model_epochs, model_lr, l2=l2, use_gpu=True, optim='adam')
acc_ = modeleval.evaluator(encoder, trainloader, testloader, noise=add_noise, print_every=100)

print('Accuracy on test set {.2f}'.format(acc_))

## Get latent features from Soccer Dataset and train a logistic regression classifier

In [ ]:
from test_soccer import run_trainer, test_img
from soccer_dataset import SoccerDataset

In [ ]:
batch_size = 50

encoder = Encoder(batch_size=batch_size)
encoder_epoch = 10
lr = 0.001
optim = 'adam'
model_name = 'AutoEncoder_lr_{}_opt_{}_epoch_{}'.format(lr, optim, encoder_epoch)
encoder.load_state_dict(load_encoder(model_name))

n_in = 32768  # 512x8x8
n_hidden = 512
n_out = 10
epochs = 20
lr = 0.01

model = LogisticRegression(n_in, n_hidden, n_out)
dataset = SoccerDataset(transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=3)
run_trainer(dataloader, encoder, model, batch_size, epochs, lr)

## Test classifier on new images

In [ ]:
test_path = 'Session6/SoccerData/test'
img1_path = test_path + '/test1.jpg'
test_img(img1_path, encoder, model)